### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 3.5.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run single_alpha_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    
    alpha = params[0]
    beta = params[1]

    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    param1=[]
    param2=[]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            #Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
            Probs[t] = update_prob(response, Probs[t], alpha)
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
#sim_data(15,[.2,2])

### New function to sim data for multiple subjects

In [9]:
def sim_mult_subject_data(n, tn, params):
    data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha', 'Beta'])
    for x in range(1, n+1):
        sdt = sim_data(tn, params)
        sdt = pd.DataFrame.from_dict(sdt)
        sdt['Subject'] = [x for t in range(0,tn*3)]
        sdt['Alpha'] = params[0]
        sdt['Beta'] = params[1]
        data = pd.concat([data,sdt])
    return data

### Create param space

In [10]:
# create parameter space
alphas = [round(x * 0.1,1) for x in range(0,11)]
betas = [x * 2 for x in range(0,11)]
combos = list(itertools.product(alphas, betas))
#for r in combos: print(r)

### Simulate data

In [11]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha', 'Beta'])
for p in range(len(combos)):
    dt = sim_mult_subject_data(10, 15, combos[p])
    dt['Subject'] = dt['Subject'] + p*10
    data = pd.concat([data, dt])
    
#data = sim_mult_subject_data(10, 15, [.2, 2])
#data
data.to_csv(path_or_buf = '../../output/simulation/sim_1alpha_model_data.csv', index = False)

### New function to fit model to multiple subjects

In [18]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'alpha', 'beta'])
    a = pd.unique(data['Subject'])
    for sub in range(1,len(a)+1):
        df = data[data['Subject']==(sub)]
        dt = df.to_dict()
        params = model_fit(dt)
        #print(params)
        line = {'Subject': sub, 'alpha': params.x[0], 'beta': params.x[1]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [19]:
precover = model_fit_mult_subject(data)
precover.to_csv(path_or_buf = '../../output/simulation/sim_1alpha_model_fits.csv', index = False)

In [20]:
print(precover)

      Subject     alpha          beta
0         1.0  1.000000  2.578735e-01
1         2.0  0.004510  2.000000e+01
2         3.0  0.000000  1.000000e-10
3         4.0  0.073109  6.110100e-01
4         5.0  0.000000  1.000000e-10
5         6.0  0.000000  1.000000e-10
6         7.0  1.000000  6.743667e-02
7         8.0  0.000000  1.000000e-10
8         9.0  0.000000  1.000000e-10
9        10.0  0.000000  1.000000e-10
10       11.0  0.000000  1.000000e-10
11       12.0  0.000000  1.000000e-10
12       13.0  0.007571  2.000000e+01
13       14.0  0.000000  1.000000e-10
14       15.0  0.007765  2.000000e+01
15       16.0  0.000000  1.000000e-10
16       17.0  0.000000  1.000000e-10
17       18.0  0.937567  1.636574e+01
18       19.0  0.000000  1.000000e-10
19       20.0  0.305456  4.579132e-01
20       21.0  0.000000  1.000000e-10
21       22.0  0.019951  2.000000e+01
22       23.0  0.000000  1.000000e-10
23       24.0  0.010899  2.000000e+01
24       25.0  0.006958  2.000000e+01
25       26.

In [ ]:
#print(precover['alpha'])

In [ ]:
#alpha = pd.Series(precover['alpha'])
#print(alpha)
#alpha.plot.hist(grid=True, bins=10, rwidth=0.9,
#                   color='#607c8e')
#plt.title('Alpha for 100 Subjects, alpha = 0.2')
#plt.xlabel('Alphas')
#plt.ylabel('Counts')
#plt.grid(axis=precover['a_gain'], alpha=0.75)

In [ ]:
#print(precover['a_loss'])

In [ ]:
#loss = pd.Series(precover['a_loss'])
#print(loss)
#loss.plot.hist(grid=True, bins=10, rwidth=0.9,
#                   color='#607c8e')
#plt.title('Alpha Loss for 100 Subjects, alpha = 0.2')
#plt.xlabel('Alphas')
#plt.ylabel('Counts')
#plt.grid(axis=precover['a_loss'], alpha=0.75)

In [ ]:
#print(precover['beta'])

In [ ]:
#beta = pd.Series(precover['beta'])
#print(beta)
#beta.plot.hist(grid=True, bins=10, rwidth=0.9,
#                   color='#607c8e')
#plt.title('Beta for 100 Subjects, beta = 1')
#plt.xlabel('Betas')
#plt.ylabel('Counts')
#plt.grid(axis=precover['beta'], beta=0.75)